# Importing the libraries

In [1]:
from langchain.llms.openai import OpenAIChat
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.globals import set_debug
from dotenv import load_dotenv

set_debug(True)  # Enable debug mode
load_dotenv()  # Load the variables from .env file

True

# Models

In this example we will use OpenAI's LLM Models to generate text, But you can use any LLM you want from the supported llms within langchain e.i. HuggingFace, LLaMa, PaLM, and etc.





Specifically we will use the newer models that are connected to `v1/chat/completion` endpoint, and can be used in `OpenAIChat` class. 

for more information about the models and the endpoint please refer to [OpenAI's API Docs](https://platform.openai.com/docs/guides/text-generation/chat-completions-vs-completions)

To setup your OpenAI API key: 
1. Create an OpenAI account
2. Create a new API key from this [link](https://platform.openai.com/api-keys)
3. Copy the API key and paste it in the `OPENAI_API_KEY` variable in `.env` file



In [2]:
# Temperature: a number between 0 and 1. The closer to 0,\
# the more predictable the text is, the closer to 1, the more Hallucinations.

model = OpenAIChat(
    model="gpt-3.5-turbo-1106",
    temperature=0.0,
)

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
# To make sure the model is working, we can test it with a prompt.

response = model("Hello, how are you?")

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Hello, how are you?"
  ]
}
[llm/end] [1:llm:OpenAIChat] [2.22s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello! I'm just a language model AI, so I don't have feelings, but I'm here to help you. How can I assist you today?",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 13,
      "completion_tokens": 32,
      "total_tokens": 45
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}


In [4]:
response

"Hello! I'm just a language model AI, so I don't have feelings, but I'm here to help you. How can I assist you today?"